In [52]:
import json

from collections import defaultdict

In [53]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)

championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []

In [54]:
#1,'9nFTht795izeoOQoIQMOHtY01trPiRW8SC7kUFIqsDJtTEeNUbpxbRriCUtJ3sApIbpwIBNV44mFjg' - Top
#2,'7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg' - Jungle
#3,'OosGFQGfn836eMPWWHKI4078RDJNVjtEN5rDdfGWkVKE9WQt57XOtBX7dJbKA2gCjvT2B2oKSOPvQA' - Mid
#4,'GDyaku8KMFbw6-NeobFdpBjk0cH7HeEwcO1O6lJDWx5DlKXBJACwQboLx_8YpwIKIjQajzWtoYjFlQ' - Bot
#5,'YXHmDXvWI5gKk7U3akJzmYR7-SUqINElADzyjLGaKOQ3uo114kFt0bgb9jfrbFyGv3wJY-fss-iGCA' - Support
#6,'a8PtUmwWlbJrEzU-nfibnfFXKmsStyk87eyureQ-juwhCH8cT9V7PIro5PogLxdME0DzYgHuuyl-Fg' - Top
#7,'735uoGC0wWXxRrYW7td31v_tqqAJkVJ3WIwoDAMnOCVvVL6gTw84KV5LFOQpwFkBdPGOQATvkx6ZHw' - Jungle
#8,'eVreMXaliubuCZDOSDs-HnMy3YSPNa3BoCKTUDDQBx3uotw36LQHmhYoJNWBxy5v0U5Wt_jV0yR80A' - Mid
#9,'esX_22maBbJu3Sf7pHod9r65uxnslhai-EnVbgIGjdC4XhW3_5Pocq561XKKt80bZlP5CPvJ3kCoow' - Bot
#10,'pnMNfdDjM97Gr0Qh67jTzEEfLefEI8qjsBxmojuIqGp7cpQPWeBBZ6jfv_UUeV3DqvUg06Rk3lphTg' - Support

In [55]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)
    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]
    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

# Example usage:
# Assuming 'targetPuuid' is the Puuid you want to find
targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")


Participant ID: 4, Champion: Varus
